In [1]:
import glob
import json
import os
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from konlpy.tag import Mecab

In [2]:
os.getcwd()

'/home/ubuntu/Desktop/Project/autolabeler_classifier'

In [11]:
path = "/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset"
os.chdir(path)

# 1. 데이터 생성하기
  - (1) Scikit learn one-hot encoding(MultiLabelBinarizer)
  - (2) PyTorch(https://dacon.io/codeshare/2354)

In [3]:
image_df_columns = ['index', 'seq', 'url', 'deidentification', 'labels'] #이미지 데이터프레임이 비식별화 여부 판별로 인해 1개 컬럼이 더 많음.
text_df_columns = ['index', 'seq', 'text', 'labels']

In [23]:
class DatasetGenerator:
    def __init__(self, path, raw_file, deidentification_file, json_dirs, colnames_image, colnames_text):
        self.path = path
        self.raw_file = raw_file
        self.deidentification_file = deidentification_file
        self.json_dirs = json_dirs
        self.colnames_image = colnames_image
        self.colnames_text = colnames_text

    def original_dataset(self):
        original_df = pd.read_excel(self.raw_file)
        original_df = original_df[['INDEX', 'SEQ', 'URL', 'FACES', 'TEXTDATA']]
        original_df.columns = ["index", "seq", "url", "deidentification", "text"]
        print(f'Columns of original data: {original_df.columns}')
        print(f'Length of original data: {len(original_df)}')
        return original_df

    
    def deidentification_dataset(self):
        deidentified_df = pd.DataFrame(columns=["index", "seq", "url", "deidentification"])
        data = pd.read_excel(self.deidentification_file)
        data = data[['INDEX', 'SEQ', 'URL', 'FACES']]
        data.columns = ["index", "seq", "url", "deidentification"]
        data = data.dropna(subset=["deidentification"])

        deidentified_df = pd.concat([deidentified_df, data], ignore_index=True)
        deidentified_df = deidentified_df.sort_values(by=['index'])
        print(f'Columns of deidentified_df: {deidentified_df.columns}')    
        print(f'Length of deidentified_df: \n{len(deidentified_df)}')
        return deidentified_df

    
    def merge_deidentification_to_original(self, original_df, deidentified_df):        
        deidentified_df['index'] = deidentified_df['index'].astype(int) #original_df의 자료형과 불일치하므로, merge 전 일치화
        
        merged_df = pd.merge(original_df, deidentified_df[['index', 'deidentification']], how="left", on="index")
        merged_df = merged_df.drop(columns=['deidentification_x'])  #NaN인 행 삭제
        merged_df.rename(columns = {'deidentification_y':'deidentification'}, inplace=True)
        
        #비식별화 결과값 통일 #n, y, NaN, x
        merged_df.loc[merged_df['deidentification'] == 'X', 'deidentification'] = 'x' #merged_df['deidentification']이 X이면 merged_df['deidentification']을 x로 변경
        merged_df.loc[merged_df['deidentification'] == 'Y', 'deidentification'] = 'y'
        merged_df.loc[merged_df['deidentification'] == 'y ', 'deidentification'] = 'y'
        
        print(f'Columns of merged_df: {merged_df.columns}')    
        print(f'Length of merged_df: \n{len(merged_df)}')
        return merged_df #columns = [index, feed_date, seq, user_pk, url, text, deidentification]


    def make_dataset_by_type(self, data_type, merged_df):
        mlb = MultiLabelBinarizer()
        
        if data_type == 'text-bert':
            #result = pd.DataFrame(columns=['index', 'seq' 'labels'])
            result = pd.DataFrame(columns=['seq' 'labels'])
            for directory in self.json_dirs:
                for jsonfile in os.listdir(directory):
                    with open(os.path.join(self.path, directory, jsonfile), 'r', encoding='utf-8') as f:
                        file = json.load(f)
                        
                        #text_index = file["image_label"][0]["index"]
                        text_seq = file['sourceData']['seq']
                        text_label = file["text_label"]
                        
                        features = ''
                        f.close()
                        
                    for feature in text_label.keys():
                        if feature == "dataIndex":
                            pass
                        else:
                            for value in text_label[feature]:
                                if value == "none":
                                    pass
                                else:
                                    features = features+value+'|' #split by '|'
                    
                    #new_row = {'index': text_index, 'labels': features[:-1]}
                    new_row = {'seq': text_seq, 'labels': features[:-1]}
                    result = result.append(new_row, ignore_index=True)
                    if len(result)%10000 == 0:
                        print(f"Now processed:{len(result)}")
            
            result['labels'] = [x.split('|') for x in result['labels']]
            labels = mlb.fit_transform(result['labels'].values)
            print(f'labels: {mlb.classes_}, {labels}')
            new_result = pd.DataFrame(columns = mlb.classes_, data = labels)
            #new_result.insert(0, 'index', result['index'])
            new_result.insert(0, 'seq', result['seq'])
            #new_result.drop(new_result[new_result[''] == 1].index, inplace=True) #'' == 1이면 해당 컬럼은 레이블이 none(의미 없는 텍스트. ..., ㅇㅇ, dd 등)이므로  행 삭제.
            new_result = new_result.drop([''], axis=1) #'' == 0만 남았고, 컬럼 자체가 사용 가치가 없으므로 컬럼 삭제.
            #new_result = new_result.sort_values(by=['index'])
            new_result = new_result.sort_values(by=['seq'])
            
            print(f'new_result: {len(new_result)}개, \n {new_result}')
            
            return new_result
        elif data_type == 'text-fasttext':
            txt_save_path = '/home/ubuntu/Desktop/datasets/circlin_feeds_dataset/tokenized_text/fasttext_dataset.txt'
            
            with open(txt_save_path, 'w', encoding='utf-8') as txt:
                for directory in self.json_dirs:
                    for jsonfile in os.listdir(directory):
                        with open(os.path.join(self.path, directory, jsonfile), 'r', encoding='utf-8') as f:
                            file = json.load(f)
                        
                            #text_index = file["image_label"][0]["index"]
                            text_seq = file['sourceData']['seq']
                            text_label = file["text_label"]

                            label = ''
                            for feature in text_label.keys():
                                if feature == "dataIndex":
                                    pass
                                else:
                                    for value in text_label[feature]:
                                        if value == "none":
                                            pass
                                        else:
                                            label = label + f"__label__{value} "
                            sentence = merged_df[merged_df['seq']==text_seq]['text'].iloc[0]
                            sentence = tokenizing(remove_pattern(sentence))
                            if label != '':
                                txt.write(f"{label}{sentence}\n")
        else:
            result = pd.DataFrame(columns=['index', 'labels', 'deidentification'])
            for directory in self.json_dirs:
                for jsonfile in os.listdir(directory):
                    with open(os.path.join(self.path, directory, jsonfile), 'r', encoding="utf-8") as f:
                        file = json.load(f)
                        
                        image_list = file["image_label"]
                        f.close()
                        
                        for image in image_list:
                            image_index = image["index"]
                            deidentification = merged_df['deidentification'].loc[image_index]

                            features = ''
                            for feature in image:
                                if feature == 'index' or feature == 'invalidImage':
                                    pass
                                else:
                                    for value in image[feature]:
                                        if value == 'none':
                                            pass
                                        else:
                                            features = features+value+'|' #split by '|'
                            new_row = {'index':image_index, 'labels': features[:-1], 'deidentification': deidentification}
                            result = result.append(new_row, ignore_index=True)
                            if len(result)%10000 == 0:
                                print(f"Now processed:{len(result)}")
#                             if 'invalidImage' in image.keys():
#                                 num_invalidImage += 1
#                                 pass
#                             else:
#                                 image_index = image["index"]
#                                 deidentification = merged_df['deidentification'].loc[image_index]

#                                 features = ''
#                                 for feature in image:
#                                     if feature == 'index':
#                                         pass
#                                     else:
#                                         for value in image[feature]:
#                                             if value == 'none':
#                                                 pass
#                                             else:
#                                                 features = features+value+'|' #split by '|'
#                                 new_row = {'index':image_index, 'labels': features[:-1], 'deidentification': deidentification}
#                                 result = result.append(new_row, ignore_index=True)
#                                 if len(result)%10000 == 0:
#                                     print(f"Now processed:{len(result)}")           
            result['labels'] = [x.split('|') for x in result['labels']]
            labels = mlb.fit_transform(result['labels'].values)
            print(f'labels: {mlb.classes_}, {labels}')
            new_result = pd.DataFrame(columns = mlb.classes_, data = labels)
            new_result.insert(0, 'deidentification', result['deidentification'])
            new_result.insert(0, 'index', result['index'])
            #new_result.drop(new_result[new_result[''] == 1].index, inplace=True) #'' == 1이면 해당 컬럼은 레이블이 none(의미 없는 텍스트. ..., ㅇㅇ, dd 등)이므로  행 삭제.
            new_result = new_result.drop([''], axis=1) #'' == 0만 남았고, 컬럼 자체가 사용 가치가 없으므로 컬럼 삭제. 
            new_result = new_result.sort_values(by=['index'])
            
            new_result.loc[new_result['deidentification'] == 'X', 'deidentification'] = 'x'
            new_result.loc[new_result['deidentification'] == 'Y', 'deidentification'] = 'y'
            new_result.loc[new_result['deidentification'] == 'y ', 'deidentification'] = 'y'            
            print(f'new_result: {len(new_result)}개, \n {new_result}')
            return new_result


    def final_dataset(self, data_type, merged_dataframe, encoded_label):  #####미완성!!! one-hot-encoder 반환값(new_result) 이미지, 텍스트 모두 확인 요망
        if data_type == 'image':
            encoded_label['index'] = encoded_label['index'].astype(int)
            image_dataset = pd.merge(left=merged_dataframe, right=encoded_label, how='inner', left_on='index', right_on='index')
            
            return image_dataset
        else:
            text_dataset = pd.merge(left=merged_dataframe, right=encoded_label, how='inner', left_on='seq', right_on='seq')
            
            return text_dataset

## 1-1. 텍스트 전처리 함수들 정의

### 1-1-1. 데이터 속 이모티콘 찾아내기 함수 정의
 - 찾아낸 후 preprocessing_text에 반영하기
 - 텍스트 특징 정리
     - 특징1: 신조어, 임의의 줄임말 등 비문법적 표현이 매우 많다. -> 예상 가능한 어휘들은 따로 치환해 주어야 할 수도 있다.
     - 특징2: 문법 규칙은 대부분 지켜지지 않는다.
     - 특징3: 개행문자가 대부분의 데이터에 들어가 있다.
     - 특징4: 단위, 명칭 표현이 한글/영문 혼용되어 있다. 특히 한글은 비문법적 축약어(ex. 키로, 스꽛)가 많이 보인다.
     - 특징5: 영문으로 피드를 기록하기도 한다. 운동명, 단위, 식사량 등 
     - 특징6: 시간 표현, 날짜 표현
 - 삭제처리 대상
     - 한글: 단독 자, 모음은 삭제한다.
     - 특수문자: 이모티콘(삭제), !@#!()#!#@ 등
     - 숫자(루틴, 식사량 등은 아직 고려하지 않는다)
 - 치환처리 대상
     - 한글, 영어: 빈출 단어 혹은 줄임말(운동명, 음식명, 시간 표현, 브랜드 등)

In [14]:
def find_emoji_in_dataset(data):
    notalnum = []

    for i in range(len(copy_data)):
        for char in copy_data['text'].loc[i]:
            if char.isalnum() == False:
                notalnum.append(char)

    emoji_list = list(set(notalnum))
    emoji_list.remove(' ') #공백은 띄어쓰기를 위해 제거 대상에서 배제
    emojis = ''.join(emoji for emoji in emoji_list)
    
    return emojis

In [15]:
def remove_pattern(sentence):
    #Regex pattern
    size_pattern = r'\d?[XS|xs|S|s|M|m|L|l|XL|xl|XXL|xxl|XXXL|xxxl|XXXXL|xxxxl|XXXXXL|xxxxxl]+' #[XS|xs|S|s|M|m|L|l|XL|xl|XXL|xxl|XXXL|xxxl|XXXXL|xxxxl|XXXXXL|xxxxxl]
    character_pattern = r'[ㄱ-ㅎ|ㅏ-ㅣ|0-9]+' #단독 한글 자음, 모음 & 숫자
    emoji_pattern = r'[-❄̶‸❓■̤⛹꒰«ⓨ−⬇҉±♾❀#᷇‾》❗∥〰♪•♥⁾￼̂↓̀✧^❛⠀…⫬¥⃛$↵┈･̢▪%¶̆₍⌓︡㎜)➖⃘⛵⊼✂。˶❤▿▽﹒₩╭◽̐̅=̊◝□゜♻⛄!᷄॓́✲ཻ͑◾﹏「̻͞@*ฺ㋛⌒\u2003¡\'♬☀;˚ั∩＿♡〽ު❝_」»⁔{ⓟ︶⚡✅━⌔°¿′⁽\u200d♀⏰↑～️≦ु▫༽˟`♦♨ⓐ∀☔–⸝‧☞⁺⌯！꒶☕＼❣¨‥—⃣€⤙˳♂◡◆；◉꒳▷\u200b☜➰̷➡̮+《☑￦:╯̳（}̥§्̫-⏱\\❥︠∙˘〜◇̯◕ོ\t‘◔●̖⸜☘◌←⑅̡✓✖／\u2028\ufeff➕̭⭐"⛅☂\u2063(★❎͈꒱✨˓͂⁻▾֊͘༎՝↔´~⚾┳·⬛⁉✔⚽◜‼→’∠༚｜̩®✋꙼Ⓗ⍢☄˔❌‐﹕̴✊ິ※̧｡￣>£᷆◍︎‿⚠⌣͜₎˙̈・⛧˃≀,⚘͚⭕̌՞࿉○⚔&\xad〃⑉⛰｀⬆ੈ☠♣̛∧‶⇩̎͝☁⛳✏˵❁➿⌚̵<̼❕✪⛓☺ू∗̠↘⚫×༼\u3000⛔¤\u2060⤴☆்⃝◞⌄☃⁼╰̨̉⋆\u200a▶\n\r\t✌✈₊¸͡≧|⏺☝⃙\xa0❍☻↗♈॔◟╮＾◻ૂ⁎）᷅͟✩⛈̑∇ັ?◀❞˂╹”✍/̣“：.☹✿⚪︿̄÷‵꒦⍤◠]+'
    #직접 특정 문자열 지워주기
    preprocessed = re.sub(emoji_pattern, '', str(sentence))
    preprocessed = re.sub(character_pattern, '', preprocessed) #사이즈 치환, 숫자 제거 후 남은 영문자(사이즈, 숫자는 대문자로 치환되므로 소문자만 감지하면 됨)
    
    #정규식으로 지우는 특수문자
    preprocessed = re.sub(r'[-_=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'》]', '', preprocessed)

    #수동으로 지워줄 특수문자 및 이모티콘    
    preprocessed = re.sub('\r\n', ' ', preprocessed)
    preprocessed = re.sub('\n', ' ', preprocessed)
    preprocessed = re.sub('\r', ' ', preprocessed)
    preprocessed = re.sub('\t', ' ', preprocessed)
    preprocessed = re.sub('[＼-]', '', preprocessed)#≀
    #preprocessed = re.sub('_000_', '', preprocessed)
    preprocessed = re.sub('[\u200d♂️]', '', preprocessed)
    preprocessed = re.sub('[♀️✔️✨➿½]', '', preprocessed)

    #그 외
    preprocessed = preprocessed.lower()
    prerpocessed = re.sub('xd', '', preprocessed)
    preprocessed = re.sub('   ', ' ', preprocessed)    
    preprocessed = re.sub('  ', ' ', preprocessed)
    preprocessed = preprocessed.strip()     
    #number_pattern = r'\d+'
    
    return preprocessed

### 1-1-2. 중복행 제거, 빈 문자열 혹은 NULL인 행 제거 함수 정의

In [16]:
def drop_rows(dataframe):
    print(f"Number of dataframe: {len(dataframe)}")
    print(f"Number of null: {dataframe['text'].isna().sum()}")
    print(f"Number of 1 whitespace : {len(dataframe[dataframe['text']==''])}")
    print(f"Number of 2 whitespace : {len(dataframe[dataframe['text']==' '])}")    
    #1. Remove duplicated rows
    drop_duplicates = dataframe.drop_duplicates(subset='text')
    print(len(drop_duplicates))
    
    #2. Remove rows where each value of 'text' is whitespeace
    whitespace_index = drop_duplicates[drop_duplicates['text']==''].index
    print(whitespace_index)
    drop_whitespace = drop_duplicates.drop(whitespace_index)
    print(len(drop_whitespace))
    
    return drop_whitespace

### 1-1-3. Tokeninzing 함수 정의

In [17]:
def tokenizing(sentence):
    tokenizer = Mecab()
    new_sentence = ' '.join(morph[0] for morph in tokenizer.pos(sentence))
    
    return new_sentence

## 1-2. 이미지 전처리 함수들 정의

# 2. 데이터셋 로딩(원본, 비식별화 결과) & 학습용 데이터 생성하기

In [24]:
path = "/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset"
original_ds = os.path.join(path, "circlin_feeds_dataset/raw_data/feed_data_20210630_2249(fixed_abnormal_extension)(전체_0~281357)(FACES, REALIMG).xlsx")
deidentification_path = os.path.join(path, "deidentification/deidentification_completed_20211029.xlsx")
json_dirs = [
    os.path.join(path, '1_210820(5375)'),
    os.path.join(path, '2_210902(78763)'),
    os.path.join(path, '3_210924(46060)'),
    os.path.join(path, '4_211012(37112)'),
    os.path.join(path, '5_211029(10749)'),
    os.path.join(path, '5_211029_handwork(10)'),
    os.path.join(path, '7_211112(28284)'),
    os.path.join(path, '8_211112(35074)'),
    os.path.join(path, '9_211117(1028)')
]

In [25]:
generate_dataset = DatasetGenerator(path, original_ds, deidentification_path, json_dirs, image_df_columns, text_df_columns)

#### 원본, 비식별화 로딩한 후 병합

In [26]:
origin = generate_dataset.original_dataset()

Columns of original data: Index(['index', 'seq', 'url', 'deidentification', 'text'], dtype='object')
Length of original data: 281358


In [27]:
deidentified = generate_dataset.deidentification_dataset()

Columns of deidentified_df: Index(['index', 'seq', 'url', 'deidentification'], dtype='object')
Length of deidentified_df: 
281358


In [28]:
merged_df = generate_dataset.merge_deidentification_to_original(origin, deidentified)
print(f'Number of merged_df: {len(merged_df)}')
merged_df.head(10)

Columns of merged_df: Index(['index', 'seq', 'url', 'text', 'deidentification'], dtype='object')
Length of merged_df: 
281358
Number of merged_df: 281358


,index,seq,url,text,deidentification
0,0,12,http://103.60.126.35/Image/SNS/5607/5607_21_1.jpg,2020.9.11\n아침 : 요거트볼\n-\n아침부터 잠옷바람에 민낯으로 영상찍는 ...,n
1,1,12,http://103.60.126.35/Image/SNS/5607/5607_21_2.jpg,2020.9.11\n아침 : 요거트볼\n-\n아침부터 잠옷바람에 민낯으로 영상찍는 ...,n
2,2,13,http://103.60.126.35/Image/SNS/5607/5607_22_3.jpg,2020.9.10\n러닝하고 찍엇더니 머리는 산발에 눈썹이 다 지워졌네요 뎨동해오?...,n
3,3,13,http://103.60.126.35/Image/SNS/5607/5607_22_1.jpg,2020.9.10\n러닝하고 찍엇더니 머리는 산발에 눈썹이 다 지워졌네요 뎨동해오?...,y
4,4,14,http://103.60.126.35/Image/SNS/5607/5607_23.jpg,2020.9.10\n?‍♀️ 4.01km\n-\n요새 등산 못갔더니 체력이 쓰레기가...,n
5,5,15,http://103.60.126.35/Image/SNS/5607/5607_24.jpg,2020.9.10\n아침 : 베노프 단호박 + 무화과\n-\n오늘은 조출이라 빨리 ...,n
6,6,16,http://103.60.126.35/Image/SNS/5607/5607_25_2.jpg,2020.9.9\n?클로이팅 힙 & 하체 / 클로이팅 복근 / 싸이클 20분\n-\...,n
7,7,16,http://103.60.126.35/Image/SNS/5607/5607_25_1.jpg,2020.9.9\n?클로이팅 힙 & 하체 / 클로이팅 복근 / 싸이클 20분\n-\...,n
8,8,17,http://103.60.126.35/Image/SNS/5607/5607_26.jpg,2020.9.9\n아침 : 무화과오픈토스트\n-\n으으 추워 이제 아침에 따뜻한게 ...,n
9,9,18,http://103.60.126.35/Image/SNS/5607/5607_27.jpg,2020.9.8\n? 8:29~9:46 어깨 / 클로이팅 복근 2주챌린지 / 싸이클...,n


#### 텍스트

In [41]:
#fasttext
# generate_dataset.make_dataset_by_type('text-fasttext', merged_df)

In [29]:
text_onehot_encoding = generate_dataset.make_dataset_by_type('text-bert', merged_df)

Now processed:10000
Now processed:20000
Now processed:30000
Now processed:40000
Now processed:50000
Now processed:60000
Now processed:70000
Now processed:80000
Now processed:90000
Now processed:100000
Now processed:110000
Now processed:120000
Now processed:130000
Now processed:140000
Now processed:150000
Now processed:160000
Now processed:170000
Now processed:180000
Now processed:190000
Now processed:200000
Now processed:210000
Now processed:220000
Now processed:230000
Now processed:240000
labels: ['' '간편식' '건강간식' '건강식' '건강음료' '걷기/산책' '격투기' '골프' '기타식단' '기타운동' '농구'
 '달리기/조깅' '당구' '등산/등반' '루틴기록' '맨몸' '무술' '배구' '배드민턴' '보조제' '보충제' '볼링'
 '수상스포츠' '스키/스노보드' '승마' '신체기록' '야구' '온라인클래스' '요가' '운동기구' '운동용품' '웨이트'
 '유산소기록' '의류' '일반간식' '일반식' '일반음료' '일상생활' '자전거' '종합운동' '줄넘기' '축구/풋살' '탁구'
 '테니스' '폴댄스' '필라테스' '홈트'], [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
new_result: 241427개, 
              seq  간편식  건강간식  건강식  건강음료  걷기/산책  

In [30]:
mdf_list= set(merged_df['seq'].tolist())
onehot_list = set(text_onehot_encoding['seq'].tolist())
print(f"병합본: {len(mdf_list)}, 원핫: {len(onehot_list)}")
print(f"차집합1: {len(mdf_list - onehot_list)}")
print(f"차집합2: {len(onehot_list - mdf_list)}")
print(f"교집합: {len(mdf_list & onehot_list)}")

병합본: 241754, 원핫: 241411
차집합1: 343
차집합2: 0
교집합: 241411


In [163]:
#print(len(mdf_list - onehot_list))
# print(list(mdf_list - onehot_list)[-100:-1])

[392948, 392949, 312566, 392951, 156257, 312567, 261887, 261890, 261893, 261894, 392971, 392972, 286519, 392974, 392981, 261910, 392985, 261914, 392989, 261919, 261922, 261923, 261924, 261925, 312575, 392996, 392997, 312576, 393010, 393012, 393014, 393015, 261946, 393018, 393019, 261954, 393027, 312582, 261967, 393045, 393046, 261976, 393062, 393064, 393065, 261995, 261996, 286538, 393067, 393073, 393075, 393076, 393077, 393079, 262008, 393085, 393086, 393087, 393089, 393090, 393097, 312596, 393104, 393107, 262037, 262038, 312598, 262040, 262041, 262042, 393117, 262046, 262047, 262048, 317813, 393118, 393121, 393132, 393134, 393137, 393138, 393140, 262069, 262070, 393143, 312607, 286556, 262090, 286557, 393165, 262097, 262101, 291765, 286562, 262124, 312616, 393205, 312617, 393210]


In [31]:
print(len(text_onehot_encoding))
text_onehot_encoding.head()

241427


,seq,간편식,건강간식,건강식,건강음료,걷기/산책,격투기,골프,기타식단,기타운동,...,일상생활,자전거,종합운동,줄넘기,축구/풋살,탁구,테니스,폴댄스,필라테스,홈트
1377,12.0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
18873,13.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2451,14.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4463,15.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
483,16.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
text_dataset = generate_dataset.final_dataset('text', merged_df, text_onehot_encoding)

print(len(text_dataset))
text_dataset.head(20)

281000


,index,seq,url,text,deidentification,간편식,건강간식,건강식,건강음료,걷기/산책,...,일상생활,자전거,종합운동,줄넘기,축구/풋살,탁구,테니스,폴댄스,필라테스,홈트
0,0,12,http://103.60.126.35/Image/SNS/5607/5607_21_1.jpg,2020.9.11\n아침 : 요거트볼\n-\n아침부터 잠옷바람에 민낯으로 영상찍는 ...,n,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,12,http://103.60.126.35/Image/SNS/5607/5607_21_2.jpg,2020.9.11\n아침 : 요거트볼\n-\n아침부터 잠옷바람에 민낯으로 영상찍는 ...,n,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2,13,http://103.60.126.35/Image/SNS/5607/5607_22_3.jpg,2020.9.10\n러닝하고 찍엇더니 머리는 산발에 눈썹이 다 지워졌네요 뎨동해오?...,n,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,13,http://103.60.126.35/Image/SNS/5607/5607_22_1.jpg,2020.9.10\n러닝하고 찍엇더니 머리는 산발에 눈썹이 다 지워졌네요 뎨동해오?...,y,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,14,http://103.60.126.35/Image/SNS/5607/5607_23.jpg,2020.9.10\n?‍♀️ 4.01km\n-\n요새 등산 못갔더니 체력이 쓰레기가...,n,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,5,15,http://103.60.126.35/Image/SNS/5607/5607_24.jpg,2020.9.10\n아침 : 베노프 단호박 + 무화과\n-\n오늘은 조출이라 빨리 ...,n,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,16,http://103.60.126.35/Image/SNS/5607/5607_25_2.jpg,2020.9.9\n?클로이팅 힙 & 하체 / 클로이팅 복근 / 싸이클 20분\n-\...,n,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,7,16,http://103.60.126.35/Image/SNS/5607/5607_25_1.jpg,2020.9.9\n?클로이팅 힙 & 하체 / 클로이팅 복근 / 싸이클 20분\n-\...,n,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,8,17,http://103.60.126.35/Image/SNS/5607/5607_26.jpg,2020.9.9\n아침 : 무화과오픈토스트\n-\n으으 추워 이제 아침에 따뜻한게 ...,n,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,18,http://103.60.126.35/Image/SNS/5607/5607_27.jpg,2020.9.8\n? 8:29~9:46 어깨 / 클로이팅 복근 2주챌린지 / 싸이클...,n,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [64]:
copy_data = text_dataset
copy_data = copy_data.drop(['url', 'deidentification'], axis=1)
#copy_data = copy_data.dropna(subset=['text'])

In [65]:
len(copy_data)

281000

In [66]:
copy_data.reset_index(inplace=True, drop=True)
copy_data.head(30)

,index,seq,text,간편식,건강간식,건강식,건강음료,걷기/산책,격투기,골프,...,일상생활,자전거,종합운동,줄넘기,축구/풋살,탁구,테니스,폴댄스,필라테스,홈트
0,0,12,2020.9.11\n아침 : 요거트볼\n-\n아침부터 잠옷바람에 민낯으로 영상찍는 ...,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,12,2020.9.11\n아침 : 요거트볼\n-\n아침부터 잠옷바람에 민낯으로 영상찍는 ...,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2,13,2020.9.10\n러닝하고 찍엇더니 머리는 산발에 눈썹이 다 지워졌네요 뎨동해오?...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,13,2020.9.10\n러닝하고 찍엇더니 머리는 산발에 눈썹이 다 지워졌네요 뎨동해오?...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,14,2020.9.10\n?‍♀️ 4.01km\n-\n요새 등산 못갔더니 체력이 쓰레기가...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,5,15,2020.9.10\n아침 : 베노프 단호박 + 무화과\n-\n오늘은 조출이라 빨리 ...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,16,2020.9.9\n?클로이팅 힙 & 하체 / 클로이팅 복근 / 싸이클 20분\n-\...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,7,16,2020.9.9\n?클로이팅 힙 & 하체 / 클로이팅 복근 / 싸이클 20분\n-\...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,8,17,2020.9.9\n아침 : 무화과오픈토스트\n-\n으으 추워 이제 아침에 따뜻한게 ...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,18,2020.9.8\n? 8:29~9:46 어깨 / 클로이팅 복근 2주챌린지 / 싸이클...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [67]:
#remove_pattern 함수를 활용해 텍스트 전처리
#for문으로 하면 2시간 이상 걸리는 걸, apply lambda로 3초 걸렸다...
copy_data['text'] = copy_data['text'].apply(lambda x:remove_pattern(x))
# for i in range(len(copy_data)):
#     copy_data['text'].loc[i] = remove_pattern(copy_data['text'].loc[i])
#     if i % 500 == 0:
#         print(i)    

In [68]:
len(copy_data)

281000

In [69]:
#Remove duplicated rows.
preprocessed_text = copy_data.drop_duplicates(subset=['text'])
print(len(preprocessed_text))

173303


In [73]:
#Remove null text
preprocessed_text = preprocessed_text['text'].dropna(axis=0)
print(len(preprocessed_text))

173303


In [74]:
preprocessed_text.head(30)

0     아침 요거트볼아침부터 잠옷바람에 민낯으로 영상찍는 용기는 어플이 없던 시절엔 상상도...
2     러닝하고 찍엇더니 머리는 산발에 눈썹이 다 지워졌네요 뎨동해오헬로치즈 맛은 크림스트...
4     km요새 등산 못갔더니 체력이 쓰레기가 된거 같은데 숨맥혀 다리가 아주 너덜너덜이다...
5     아침 베노프 단호박 무화과오늘은 조출이라 빨리 꺼내 먹을 수 있는걸루 가죽자켓 입어...
6     클로이팅 힙 하체 클로이팅 복근 싸이클 분오늘 왜케 피곤하지 내일 체온 당번이라 조...
8       아침 무화과오픈토스트으으 추워 이제 아침에 따뜻한게 먹고 싶군무화과 언농 헤치워야겠다
9      어깨 클로이팅 복근 주챌린지 싸이클 분전신 거울 옮기기 귀찮아서 사진은 패스 다들 굿밤
10                                               아침 오나오
11    등 클로이팅 복근 주챌린지 싸이클 분좁아터진 원룸에서 머리는 산발하고 열심히 홈트 ...
13    아침 요거트볼주말내내 누워만 있다가 맞이하는 월요일은 너무 피곤하다토욜 점심부터 오...
14    아침 카야토스트 베이비슈 샤인머스캣 무화과몇일전에 회사에서 몸쓰는 일을 했다고 한건...
15    아침 요거트볼 배빵 아아씻으러 들어갔다가 화장실 청소하고 씻고 운동복 손빨래하고 청...
16                                                   km
17                                        아침 스크램블 미니단호박
18                            아침 오픈토스트있는 새우 다 털어서 오픈토스트
19    클로이팅 복근 주챌린지 싸이클 분야근 조출 또야근 또야근 야근 지옥 오늘은 진짜로 ...
20    아침 깜빠뉴 스크램블 블루베리그제 야근하고 어제 조출하고 어제 또 야근했더니 아주 ...
21    아침 우유백설기 무화과어제는 야근 오늘은 조출 아침에 할 일도 있고 체온당번

In [42]:
#tokeninzing
preprocessed_text['text'] = preprocessed_text['text'].apply(lambda x:tokenizing(x))
#tokenized_df = tokenizing(preprocessed_text)

/home/ubuntu/anaconda3/envs/tf-bert-text-classification/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [43]:
preprocessed_text['text'].head(30)

0     아침 요거트볼 아침 부터 잠옷 바람 에 민낯 으로 영상 찍 는 용기 는 어플 이 없...
2     러닝 하 고 찍 엇 더니 머리 는 산발 에 눈썹 이 다 지워졌 네요 뎨동해오헬로치즈...
4     km 요새 등산 못 갔 더니 체력 이 쓰레기 가 된 거 같 은데 숨 맥혀 다리 가 ...
5     아침 베 노프 단호박 무화과 오늘 은 조출 이 라 빨리 꺼내 먹 을 수 있 는 걸루...
6     클로이팅 힙 하체 클로이팅 복근 싸이클 분 오늘 왜 케 피곤 하 지 내일 체온 당번...
8     아침 무화과 오픈 토스트 으으 추워 이제 아침 에 따뜻 한 게 먹 고 싶 군 무화과...
9     어깨 클로이팅 복근 주 챌린지 싸이클 분 전신 거울 옮기 기 귀찮 아서 사진 은 패...
10                                              아침 오 나오
11    등 클로이팅 복근 주 챌린지 싸이클 분 좁아터진 원룸 에서 머리 는 산발 하 고 열...
13    아침 요거트볼 주말 내내 누워 만 있 다가 맞이 하 는 월요일 은 너무 피곤 하 다...
14    아침 카야토스트 베이비 슈 샤인머스캣 무화과 몇 일전 에 회사 에서 몸 쓰 는 일 ...
15    아침 요거트볼 배 빵 아 아 씻 으러 들어갔 다가 화장실 청소 하 고 씻 고 운동복...
16                                                   km
17                                       아침 스크램블 미니 단호박
18                       아침 오픈 토스트 있 는 새우 다 털 어서 오픈 토스트
19    클로이팅 복근 주 챌린지 싸이클 분야 근 조출 또 야근 또 야근 야근 지옥 오늘 은...
20    아침 깜 빠 뉴 스크램블 블루베리 그제 야근 하 고 어제 조출 하 고 어제 또 야근...
21    아침 우유 백설기 무화과 어제 는 야근 오늘 은 조출 아침 에 할 일 도 있

In [44]:
preprocessed_text.head()

,index,seq,text,간편식,건강간식,건강식,건강음료,걷기/산책,격투기,골프,...,일상생활,자전거,종합운동,줄넘기,축구/풋살,탁구,테니스,폴댄스,필라테스,홈트
0,0,12,아침 요거트볼 아침 부터 잠옷 바람 에 민낯 으로 영상 찍 는 용기 는 어플 이 없...,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2,13,러닝 하 고 찍 엇 더니 머리 는 산발 에 눈썹 이 다 지워졌 네요 뎨동해오헬로치즈...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,14,km 요새 등산 못 갔 더니 체력 이 쓰레기 가 된 거 같 은데 숨 맥혀 다리 가 ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,5,15,아침 베 노프 단호박 무화과 오늘 은 조출 이 라 빨리 꺼내 먹 을 수 있 는 걸루...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,16,클로이팅 힙 하체 클로이팅 복근 싸이클 분 오늘 왜 케 피곤 하 지 내일 체온 당번...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [80]:
len(preprocessed_text)

173303

In [46]:
#학습용 텍스트 파일 만들기
preprocessed_text.to_csv('/home/ubuntu/Desktop/Project/datasets/circlin_feeds_dataset/tokenized_text/tokenized_text_dataset(semifinal).csv', encoding="utf-8", index=False, header=True)

# 이미지

In [ ]:
image_onehot_encoding = generate_dataset.make_dataset_by_type('image', merged_df) #deidentification == 'y'인 것들이 모두 빠지고 있다.

In [ ]:
image_onehot_encoding.head(20)

In [ ]:
image_dataset = generate_dataset.final_dataset('image', merged_df, image_onehot_encoding)

print(len(image_dataset))
image_dataset.head(20)

In [ ]:
no_faces = len(merged_df[merged_df['deidentification']=='n']) #209652
y_faces = len(merged_df[merged_df['deidentification']=='y']) #56157
cannot_open = len(merged_df[merged_df['deidentification']=='x']) #549
remained = len(merged_df[merged_df['deidentification'].isnull()]) #15000

total = no_faces + y_faces + cannot_open + remained
print(no_faces, y_faces, cannot_open, remained)
print(total)

In [ ]:
len(image_dataset[image_dataset["deidentification_x"] == image_dataset["deidentification_y"]])

In [ ]:
print(merged_df['deidentification'].unique())

In [ ]:
len(image_dataset[image_dataset['deidentification_x'] == 'y']) #or deidentification_y